In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl

# df = pd.read_csv("../data/raw/us_stocks_sip/minute_aggs_v1/2025/08/2025-08-11.csv")
# df = df[df['ticker'] =='AAPL']
stock_path = "../data/lake/us_stocks_sip/minute_aggs_v1/2025/08/2025-08-08.parquet"

df = pl.read_parquet(stock_path).filter(pl.col("ticker") == 'AAPL')

print(df.head())

print(df.count())
# 计算一日的vprice
vprice_day = (df['volume'] * df['close']).sum() / df['volume'].sum()

# print("收盘价Close:", df.iloc[-1]['close'])
print("一日的vprice:", vprice_day)


In [ ]:
import ipywidgets as widgets

mode = widgets.Dropdown(
    options=['backtest', 'live'],
    value='backtest',
    description='Mode:',
)

prefix = widgets.Text(
    value='demo',
    description='Prefix:',
)

display(mode, prefix)

def run_strategy(m, p):
    print(f"运行模式: {m}, 前缀: {p}")

widgets.interactive(run_strategy, m=mode, p=prefix)


In [ ]:
import glob
import polars as pl
from datetime import datetime, timedelta

# 数据路径（7月份所有 parquet 文件）
# valid columns: ["ticker", "volume", "open", "close", "high", "low", "window_start", "transactions"]
data_dir = "../data/lake/us_stocks_sip/minute_aggs_v1/2025/07/*.parquet"

lf = (
    pl.scan_parquet(data_dir)
    .filter(pl.col("ticker") == "AAPL")
    .with_columns(
        pl.from_epoch(pl.col("window_start"), time_unit='ns')
        # .dt.convert_time_zone("America/New_York")
        .alias('datetime')
    )
)

df_all = lf.collect(engine='streaming')

# 计算整月的成交量加权价格（VWAP）
vprice_month = (df_all["volume"] * df_all["close"]).sum() / df_all["volume"].sum()
print(f"\n2025-07 VWAP: {vprice_month}")


# 使用Polars检查周末数据
print("检查周末数据...")
week_data = df_all.filter(
    (pl.col("datetime") >= pl.datetime(2025, 7, 5)) & 
    (pl.col("datetime") <= pl.datetime(2025, 7, 6, 23, 59, 59))
)
print(f"2025-07-05到2025-07-06的数据行数: {week_data.height}")

# 更详细的日期检查
print("\n详细的日期检查:")
dates = ['2025-07-04', '2025-07-05', '2025-07-06', '2025-07-07']
for date_str in dates:
    year, month, day = map(int, date_str.split('-'))
    start_datetime = pl.datetime(year, month, day)
    end_datetime = pl.datetime(year, month, day, 23, 59, 59)
    
    date_data = df_all.filter(
        (pl.col("datetime") >= start_datetime) & 
        (pl.col("datetime") <= end_datetime)
    )
    
    # 获取星期几
    weekday_num = datetime(year, month, day).weekday()  # 0=Monday, 6=Sunday
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday = weekdays[weekday_num]
    
    print(f"{date_str} ({weekday}): {date_data.height} 行数据")

# 使用Polars检查星期几的分布
print("\n按星期几统计数据量:")
df_with_weekday = df_all.with_columns(
    pl.col("datetime").dt.strftime("%A").alias("weekday")
)
weekday_counts = df_with_weekday.group_by("weekday").agg(pl.len().alias("count")).sort("count", descending=True)
print(weekday_counts)


In [ ]:
# use yfinance to double check splits discrepancy
import yfinance as yf
import pandas as pd

ticker ='ENVX'

df = yf.download(ticker,period='10y',actions=True)
df= pd.DataFrame(df)
with pd.option_context('display.max_rows', 50, 
                       'display.max_columns', 20, 
                       'display.max_colwidth', 100,
                       'display.width', None,           # 不限制总宽度
                       'display.expand_frame_repr', False):  # 不要换行显示
    df = df[df['Stock Splits'] > 0].dropna()
    print(df)

In [ ]:
def apply_split_adjustments(df: pl.DataFrame, splits: pl.DataFrame, price_columns: list = None) -> pl.DataFrame:
    """
    通用的分拆调整函数
    
    Args:
        df: 包含价格数据的DataFrame，必须包含 'ticker' 和 'datetime' 列
        splits: 分拆数据DataFrame，包含 'ticker', 'execution_date', 'split_from', 'split_to' 列
        price_columns: 需要调整的价格列名列表，默认为 ['open', 'close', 'high', 'low']
    
    Returns:
        调整后的DataFrame
    """
    if price_columns is None:
        price_columns = ['open', 'close', 'high', 'low']
    
    # 确保日期格式正确
    splits_processed = splits.with_columns([
        pl.col('execution_date').str.to_date().alias('split_date'),
        (pl.col('split_from') / pl.col('split_to')).alias('split_ratio')
    ])
    
    # 为每个ticker计算累计分拆比率
    result_df = df.clone()
    
    for ticker in df['ticker'].unique():
        ticker_splits = splits_processed.filter(pl.col('ticker') == ticker).sort('split_date')
        ticker_data = df.filter(pl.col('ticker') == ticker).sort('datetime')
        
        if ticker_splits.height == 0:
            continue
            
        # 为每行数据计算需要应用的累计分拆比率
        ticker_data = ticker_data.with_columns([
            pl.col('datetime').dt.date().alias('data_date')
        ])
        
        # 使用join_asof进行时间匹配，获取每个数据点之后发生的所有分拆
        adjusted_data = ticker_data.clone()
        
        for price_col in price_columns:
            if price_col in ticker_data.columns:
                # 计算该日期之后的所有分拆的累计比率
                cumulative_ratios = []
                
                for row in ticker_data.iter_rows(named=True):
                    data_date = row['data_date']
                    # 获取该日期之后的所有分拆
                    future_splits = ticker_splits.filter(pl.col('split_date') > data_date)
                    
                    # 计算累计分拆比率
                    if future_splits.height > 0:
                        cumulative_ratio = future_splits['split_ratio'].product()
                    else:
                        cumulative_ratio = 1.0
                    
                    cumulative_ratios.append(cumulative_ratio)
                
                # 应用分拆调整
                adjusted_data = adjusted_data.with_columns([
                    (pl.col(price_col) * pl.Series(cumulative_ratios)).alias(price_col)
                ])
        
        # 调整成交量（分拆时成交量按相反比例调整）
        if 'volume' in ticker_data.columns:
            volume_ratios = []
            for row in ticker_data.iter_rows(named=True):
                data_date = row['data_date']
                future_splits = ticker_splits.filter(pl.col('split_date') > data_date)
                
                if future_splits.height > 0:
                    # 成交量按分拆比率的倒数调整
                    volume_ratio = (1 / future_splits['split_ratio']).product()
                else:
                    volume_ratio = 1.0
                
                volume_ratios.append(volume_ratio)
            
            adjusted_data = adjusted_data.with_columns([
                (pl.col('volume') * pl.Series(volume_ratios)).alias('volume')
            ])
        
        # 更新结果
        result_df = result_df.filter(pl.col('ticker') != ticker).vstack(
            adjusted_data.drop('data_date')
        )
    
    return result_df.sort(['ticker', 'datetime'])

In [1]:
from quant101.core_2.config import all_tickers_dir
import os
import glob

end_date = "2025-08-01"

file_date = end_date.replace('-', '')
all_tickers_file = os.path.join(all_tickers_dir, f'all_tickers_{file_date}.parquet')

if not os.path.exists(all_tickers_file):
    # Find all matching parquet files
    files = glob.glob(os.path.join(all_tickers_dir, 'all_tickers_*.parquet'))
    # Extract date part and sort by closest, prefer larger or equal dates
    def extract_date(f):
        try:
            return int(os.path.basename(f).split('_')[-1].replace('.parquet', ''))
        except Exception:
            return float('inf')
    file_dates = [(f, extract_date(f)) for f in files]
    file_dates = [fd for fd in file_dates if fd[1] != float('inf')]
    if file_dates:
        # Sort by (abs diff, prefer larger date)
        file_dates.sort(key=lambda x: (abs(x[1] - int(file_date)), x[1] < int(file_date)))
        all_tickers_file = file_dates[0][0]
    else:
        all_tickers_file = None
        
print(f"Using all_tickers_file: {all_tickers_file}")

Using all_tickers_file: /mnt/blackdisk/quant_data/polygon_data/raw/us_stocks_sip/us_all_tickers/all_tickers_20250904.parquet


In [ ]:
import polars as pl
from quant101.utils.compute import calculate_signal_duration
import datetime as dt

# 定义每个 ticker 的时间序列
ticker_data = {
    "A": [
        dt.datetime(2023,1,5),
        dt.datetime(2023,1,6),
        dt.datetime(2024,5,3),
        dt.datetime(2024,10,15),
    ],
    "AA": [
        dt.datetime(2023,5,22),
        dt.datetime(2025,6,2),
        dt.datetime(2025,6,25),
        dt.datetime(2025,6,26),
        dt.datetime(2025,6,27),
    ],
    "AACB": [
        dt.datetime(2025,6,26),
        dt.datetime(2025,6,27),
        dt.datetime(2025,6,30),
        dt.datetime(2025,7,1),
        dt.datetime(2025,7,2),
        dt.datetime(2025,7,3),
    ],
}

# 每个 ticker 转成一个 DataFrame，然后拼接
dfs = []
for ticker, dates in ticker_data.items():
    df = pl.DataFrame(
        {
            "timestamps": dates,
            "ticker": [ticker] * len(dates),
            "signal": [1] * len(dates),
        }
    )
    dfs.append(df)

signals = pl.concat(dfs)

with pl.Config(tbl_cols=20, tbl_rows=500, tbl_width_chars=1000, verbose=True):
    # print(signals)

    signal_durations, avg_durations = calculate_signal_duration(signals)
    print("每个ticker的信号持续期详情:")
    print(signal_durations.head(20))
    print("\n每个ticker的平均持续天数:")
    print(avg_durations.head(20))
                
    # 也可以查看整体统计
    print(f"\n整体统计:")
    print(f"平均信号持续天数: {avg_durations['avg_duration_days'].mean():.2f} 天")
    print(f"中位数信号持续天数: {avg_durations['avg_duration_days'].median():.2f} 天")
    print(f"最长平均持续天数: {avg_durations['avg_duration_days'].max():.2f} 天")
    print(f"最短平均持续天数: {avg_durations['avg_duration_days'].min():.2f} 天")

每个ticker的信号持续期详情:
shape: (8, 6)
┌────────┬──────────────┬─────────────────────┬─────────────────────┬──────────────┬───────────────┐
│ ticker ┆ signal_group ┆ start_date          ┆ end_date            ┆ signal_count ┆ duration_days │
│ ---    ┆ ---          ┆ ---                 ┆ ---                 ┆ ---          ┆ ---           │
│ str    ┆ u32          ┆ datetime[μs]        ┆ datetime[μs]        ┆ u32          ┆ i64           │
╞════════╪══════════════╪═════════════════════╪═════════════════════╪══════════════╪═══════════════╡
│ A      ┆ 1            ┆ 2023-01-05 00:00:00 ┆ 2023-01-06 00:00:00 ┆ 2            ┆ 2             │
│ A      ┆ 2            ┆ 2024-05-03 00:00:00 ┆ 2024-05-03 00:00:00 ┆ 1            ┆ 1             │
│ A      ┆ 3            ┆ 2024-10-15 00:00:00 ┆ 2024-10-15 00:00:00 ┆ 1            ┆ 1             │
│ AA     ┆ 1            ┆ 2023-05-22 00:00:00 ┆ 2023-05-22 00:00:00 ┆ 1            ┆ 1             │
│ AA     ┆ 2            ┆ 2025-06-02 00:00:00 ┆ 2025-06-02 

keys/aggregates are not partitionable: running default HASH AGGREGATION
keys/aggregates are not partitionable: running default HASH AGGREGATION
